USING QSVC IN ORDER TO TEST STANDARD QUANTUM KERNEL APPROACH

In [30]:
#needed to import qke procedures
import sys
from pathlib import Path

main_dir = str(Path().resolve().parent)
sys.path.insert(1, main_dir)

In [21]:
#scikit learn and tools
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
#load dataset with panda
#data are scaled outside the notebook
env = pd.read_csv('../data/env.sel3.scaled.csv') 
#take a look of the dataset
env.describe() 

,illuminance,blinds,lamps,rh,co2,temp,occupancy
count,2.865000e+03,2865.000000,2865.000000,2865.000000,2.865000e+03,2.865000e+03,2865.000000
mean,-3.571314e-16,0.801622,0.596464,0.473229,-5.952190e-16,-7.142629e-16,-0.130192
std,1.000000e+00,0.372305,0.489316,0.080185,1.000000e+00,1.000000e+00,0.991662
min,-1.016287e+00,0.000000,0.000000,0.243300,-1.222685e+00,-1.774534e+00,-1.000000
25%,-8.575745e-01,1.000000,0.000000,0.433700,-8.492006e-01,-8.538436e-01,-1.000000
50%,-6.989177e-02,1.000000,1.000000,0.491100,-3.252814e-01,-1.940149e-01,-1.000000
75%,3.357060e-01,1.000000,1.000000,0.524100,7.569402e-01,7.266751e-01,1.000000
max,8.559349e+00,1.000000,1.000000,0.633300,3.583699e+00,2.614091e+00,1.000000


In [23]:
#DEFINE design matrix
Y = env['occupancy']
#X = env[['illuminance', 'blinds','lamps','co', 'rh', 'co2', 'temp']]
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X.head()

,illuminance,blinds,lamps,rh,co2,temp
0,-0.992774,1.0,0.0,0.4069,-1.001777,-0.061027
1,-0.992774,1.0,0.0,0.4069,-0.892611,-0.061027
2,-0.992774,1.0,0.0,0.4069,-1.002203,-0.061027
3,-0.992774,1.0,0.0,0.4069,-0.867245,-0.061027
4,-0.992774,1.0,0.0,0.4069,-0.958793,-0.061027


In [24]:
#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y)
#sanity check
print(f'TRAIN***** {X_train.shape}')
print(f'TEST***** {X_test.shape}')

TRAIN***** (2148, 6)
TEST***** (717, 6)


In [45]:
#define fidelity
from qiskit_machine_learning.kernels import FidelityStatevectorKernel 

from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms.classifiers import QSVC

fm = ZZFeatureMap(feature_dimension=X_train.shape[1])
#fm = Circuits.x_encoded(n_wire=X_train.shape[1])

q_kernel = FidelityStatevectorKernel(feature_map=fm)
#q_kernel = FidelityQuantumKernel(feature_map=fm)

qsvc = QSVC(quantum_kernel=q_kernel)


In [46]:
#trainingprediction...
qsvc.fit(X_train, y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_statevector_kernel.FidelityStatevectorKernel object at 0x791d817954f0>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [47]:
from sklearn.metrics import accuracy_score

#result...
predictions = qsvc.predict(X_test)
accuracy_score(predictions, y_test)



0.8242677824267782